In [21]:
%pip install -r ~/code/GitHub/UPennContrast/devops/girder/annotation_client/requirements.txt
%pip install -e ~/code/GitHub/UPennContrast/devops/girder/annotation_client/
%pip install pandas
%pip install imageio
%pip install rtree
%pip install scikit-image
%pip install shapely
%pip install nd2reader
%pip install geopandas

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/arjunraj/code/GitHub/UPennContrast/devops/girder/annotation_client
  Attempting uninstall: annotation-client
    Found existing installation: annotation-client 0.1
    Uninstalling annotation-client-0.1:
      Successfully uninstalled annotation-client-0.1
  Running setup.py develop for annotation-client
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading 

In [22]:
import girder_client
import annotation_client.annotations as annotations
import annotation_client.tiles as tiles
import pandas as pd
import annotation_tools
#import imageio
from shapely.geometry import Point, Polygon, LineString
#pd.set_option('display.max_colwidth', None)
import numpy as np

In [23]:
apiUrl = 'http://localhost:8080/api/v1'
username = 'admin'
password = 'password'

In [24]:
client = girder_client.GirderClient(apiUrl=apiUrl)
user_id = client.authenticate(username=username, password=password)['_id']
annotationClient = annotations.UPennContrastAnnotationClient(apiUrl=apiUrl, token=client.token)

In [25]:
datasets = dict()
for folder in annotationClient.client.listFolder(user_id,'user'):
    for dataset in annotationClient.client.listFolder(folder['_id']):
        datasets[dataset['name']] = dataset['_id']

pd.DataFrame(datasets.keys(),columns=['Dataset Name'])

,Dataset Name
0,111111_20210723_155544_798__Plate20210723_STR_...
1,111_20200110_2Dent_001
2,20200110_2Dent_001_maxmerge
3,"20221122_143426_960__WellA4_ChannelDAPI,CY3,A5..."
4,"20230314_175616_594__WellB2_ChannelDAPI,YFP,CY..."
...,...
163,UNROLL_MS2_1_maxmerge
164,Well2_50framesCOMPOSITENORECODE
165,Well2_50framesCOMPOSITENORECODE_maxmerge
166,"WellA1_ChannelBrightfield,YFP,CY5,DAPI,A594_Se..."


In [26]:
#dataset_name = '300cell-gastruloids_E11_1_04d16h00m'
dataset_name = 'SCALE_TESTnormmedia_8well_col2_livecellgfp'

In [27]:
datasetId = datasets[dataset_name]
datasetClient = tiles.UPennContrastDataset(apiUrl=apiUrl, token=client.token, datasetId=datasetId)
print(datasetClient.tiles)
tile_width = datasetClient.tiles['tileWidth']
tile_height = datasetClient.tiles['tileHeight']
print(tile_width)
print(tile_height)

{'IndexRange': {'IndexC': 2, 'IndexT': 4, 'IndexXY': 6, 'IndexZ': 7}, 'IndexStride': {'IndexC': 1, 'IndexT': 14, 'IndexXY': 56, 'IndexZ': 2}, 'bandCount': 1, 'channelmap': {'Brightfield': 0, 'YFP': 1}, 'channels': ['Brightfield', 'YFP'], 'dtype': 'uint16', 'frames': [{'Channel': 'Brightfield', 'Frame': 0, 'Index': 0, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 0}, {'Channel': 'YFP', 'Frame': 1, 'Index': 0, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 0}, {'Channel': 'Brightfield', 'Frame': 2, 'Index': 1, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 1}, {'Channel': 'YFP', 'Frame': 3, 'Index': 1, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 1}, {'Channel': 'Brightfield', 'Frame': 4, 'Index': 2, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 2}, {'Channel': 'YFP', 'Frame': 5, 'Index': 2, 'IndexC': 1, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 2}, {'Channel': 'Brightfield', 'Frame': 6, 'Index': 3, 'IndexC': 0, 'IndexT': 0, 'IndexXY': 0, 'IndexZ': 3}, {'Channel': 'YFP', 'F

In [38]:
# May need to change the limit for large numbers of annotations. Default is 50.
pointAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, limit = 1000000, shape='point')
blobAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, limit = 1000000, shape='polygon')
lineAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, limit = 1000000, shape='line')
#allAnnotationList = annotationClient.getAnnotationsByDatasetId(datasetId, limit = 1000000)
allAnnotationList = pointAnnotationList + blobAnnotationList + lineAnnotationList

In [39]:
import pprint
print(datasetId)
connectionList = annotationClient.getAnnotationConnections(datasetId)
pprint.pprint(connectionList,indent=2)

64ff0d8ce7761ae25b62da75
[]


In [40]:
print(len(blobAnnotationList))
print(len(pointAnnotationList))
print(len(allAnnotationList))
#print(len(lineAnnotationList))
print(blobAnnotationList[0])
print(pointAnnotationList[0])
#print(lineAnnotationList[0])

13
86
99
{'_id': '64ff2f5ee7761ae25b62e06d', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 263, 'y': 390}, {'x': 261, 'y': 385}, {'x': 258, 'y': 379}, {'x': 250, 'y': 381}, {'x': 231, 'y': 378}, {'x': 220, 'y': 377}, {'x': 218, 'y': 382}, {'x': 218, 'y': 389}, {'x': 220, 'y': 394}, {'x': 226, 'y': 402}, {'x': 231, 'y': 405}, {'x': 239, 'y': 407}, {'x': 248, 'y': 408}, {'x': 258, 'y': 408}, {'x': 263, 'y': 407}, {'x': 265, 'y': 402}], 'datasetId': '64ff0d8ce7761ae25b62da75', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'polygon', 'tags': ['nucleus']}
{'_id': '64ff2f67e7761ae25b62e076', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 0, 'coordinates': [{'x': 227, 'y': 354, 'z': 0}], 'datasetId': '64ff0d8ce7761ae25b62da75', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'point', 'tags': ['spot']}


In [41]:
# Ignore this block.
pointList1 = annotation_tools.get_annotations_with_tags(allAnnotationList,['myPoint'],exclusive=False)
pointList2 = annotation_tools.get_annotations_with_tags(allAnnotationList,['yourPoint'],exclusive=False)
blobList1 = annotation_tools.get_annotations_with_tags(allAnnotationList,['myBlob'],exclusive=False)
print(len(pointList1))
print(len(pointList2))
print(len(blobList1))

0
0
0


In [42]:
spotList = annotation_tools.get_annotations_with_tags(allAnnotationList,['spot'],exclusive=False)
rnaList = annotation_tools.get_annotations_with_tags(allAnnotationList,['RNA'],exclusive=False)
nucleusList = annotation_tools.get_annotations_with_tags(allAnnotationList,['nucleus'],exclusive=False)
print(len(spotList))
print(len(rnaList))
print(len(nucleusList))


50
36
13


In [49]:
parentTag = ['nucleus']
childTag = ['RNA']

parentList = annotation_tools.get_annotations_with_tags(allAnnotationList,parentTag,exclusive=False)
childList = annotation_tools.get_annotations_with_tags(allAnnotationList,childTag,exclusive=False)
print(parentList[0])
print(childList[0])

{'_id': '64ff2f5ee7761ae25b62e06d', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 1, 'coordinates': [{'x': 263, 'y': 390}, {'x': 261, 'y': 385}, {'x': 258, 'y': 379}, {'x': 250, 'y': 381}, {'x': 231, 'y': 378}, {'x': 220, 'y': 377}, {'x': 218, 'y': 382}, {'x': 218, 'y': 389}, {'x': 220, 'y': 394}, {'x': 226, 'y': 402}, {'x': 231, 'y': 405}, {'x': 239, 'y': 407}, {'x': 248, 'y': 408}, {'x': 258, 'y': 408}, {'x': 263, 'y': 407}, {'x': 265, 'y': 402}], 'datasetId': '64ff0d8ce7761ae25b62da75', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'polygon', 'tags': ['nucleus']}
{'_id': '64ff2f7be7761ae25b62e09d', 'access': {'groups': [], 'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]}, 'channel': 0, 'coordinates': [{'x': 506, 'y': 743.299349619042, 'z': 0}], 'datasetId': '64ff0d8ce7761ae25b62da75', 'location': {'Time': 0, 'XY': 0, 'Z': 0}, 'shape': 'point', 'tags': ['RNA']}


In [63]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree
import numpy as np


def extract_spatial_annotation_data(obj_list):
    data = []
    for obj in obj_list:
        x, y = None, None
        shape = obj['shape']
        coords = obj['coordinates']
        
        if shape == 'point':
            x, y = coords[0]['x'], coords[0]['y']
        elif shape == 'polygon':
            polygon = Polygon([(pt['x'], pt['y']) for pt in coords])
            centroid = polygon.centroid
            x, y = centroid.x, centroid.y
        
        data.append({
            '_id': obj['_id'],
            'x': x,
            'y': y,
            'Time': obj['location']['Time'],
            'XY': obj['location']['XY'],
            'Z': obj['location']['Z']
        })
    return data

# Extract data and create dataframes
child_data = extract_spatial_annotation_data(childList)
parent_data = extract_spatial_annotation_data(parentList)

child_df = pd.DataFrame(child_data)
parent_df = pd.DataFrame(parent_data)

gdf_child = gpd.GeoDataFrame(child_df, geometry=gpd.points_from_xy(child_df.x, child_df.y))
gdf_parent = gpd.GeoDataFrame(parent_df, geometry=gpd.points_from_xy(parent_df.x, parent_df.y))

def compute_nearest_child_to_parent(child_df, parent_df, groupby_cols=['Time', 'XY', 'Z'], max_distance=None):
    # Empty DataFrame to store results
    child_to_parent = pd.DataFrame(columns=['child_id', 'nearest_parent_id'])

    # Group by unique location combinations
    for values, group in child_df.groupby(groupby_cols):
        
        # Build a dynamic query string
        query_str = ' & '.join([f"{col} == {val}" for col, val in zip(groupby_cols, values)])
        
        # Filter parent dataframe by the same location values
        parent_group = parent_df.query(query_str)
        
        # Ensure there are parents in the group to compare to
        if parent_group.empty:
            continue
        
        # Create a cKDTree for the parent group
        tree = cKDTree(np.array(list(zip(parent_group.geometry.x, parent_group.geometry.y))))
        
        # Compute distance and index for the nearest parent for each child within this group
        distances, indices = tree.query(np.array(list(zip(group.geometry.x, group.geometry.y))))
        
        # If max_distance is provided, filter out entries beyond that distance
        if max_distance is not None:
            valid_indices = distances <= max_distance
            distances = distances[valid_indices]
            indices = indices[valid_indices]
            group = group.iloc[valid_indices]  # Update the group DataFrame to only contain valid entries
        
        # Map child IDs to nearest parent IDs for this group
        temp_df = pd.DataFrame({
            'child_id': group['_id'].values,
            'nearest_parent_id': parent_group.iloc[indices]['_id'].values
        })
        
        # Append the results to the main DataFrame
        child_to_parent = pd.concat([child_to_parent, temp_df], ignore_index=True)

    return child_to_parent



# Example usage
max_distance = 70
#child_to_parent = compute_nearest_child_to_parent(gdf_child, gdf_parent, groupby_cols=['Time', 'XY'])
child_to_parent = compute_nearest_child_to_parent(gdf_child, gdf_parent, groupby_cols=['Time', 'XY'], max_distance=max_distance)
print(len(child_to_parent))


26


In [64]:
myNewConnections = []
combined_tags = list(set(parentTag + childTag))

for index, row in child_to_parent.iterrows():
    child_id = row['child_id']
    parent_id = row['nearest_parent_id']
    
    myNewConnections.append({
        'datasetId': datasetId,  # assuming you've already set this variable elsewhere in your code
        'parentId': parent_id,
        'childId': child_id,
        'tags': combined_tags
    })

pprint.pprint(myNewConnections,indent=2)
# for connection in myNewConnections:
#     annotationClient.createConnection(connection)

annotationClient.createMultipleConnections(myNewConnections)


[ { 'childId': '64ff2f7ce7761ae25b62e0a3',
    'datasetId': '64ff0d8ce7761ae25b62da75',
    'parentId': '64ff2f64e7761ae25b62e073',
    'tags': ['nucleus', 'RNA']},
  { 'childId': '64ff2f7de7761ae25b62e0a6',
    'datasetId': '64ff0d8ce7761ae25b62da75',
    'parentId': '64ff2f64e7761ae25b62e073',
    'tags': ['nucleus', 'RNA']},
  { 'childId': '64ff2f7ee7761ae25b62e0a9',
    'datasetId': '64ff0d8ce7761ae25b62da75',
    'parentId': '64ff2f64e7761ae25b62e073',
    'tags': ['nucleus', 'RNA']},
  { 'childId': '64ff2f7ee7761ae25b62e0ac',
    'datasetId': '64ff0d8ce7761ae25b62da75',
    'parentId': '64ff2f64e7761ae25b62e073',
    'tags': ['nucleus', 'RNA']},
  { 'childId': '64ff2f7fe7761ae25b62e0af',
    'datasetId': '64ff0d8ce7761ae25b62da75',
    'parentId': '64ff2f60e7761ae25b62e070',
    'tags': ['nucleus', 'RNA']},
  { 'childId': '64ff2f80e7761ae25b62e0b2',
    'datasetId': '64ff0d8ce7761ae25b62da75',
    'parentId': '64ff2f60e7761ae25b62e070',
    'tags': ['nucleus', 'RNA']},
  { 'child

[{'_id': '6501d9b127b4271da176f2f1',
  'access': {'groups': [],
   'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
  'childId': '64ff2f7ce7761ae25b62e0a3',
  'datasetId': '64ff0d8ce7761ae25b62da75',
  'parentId': '64ff2f64e7761ae25b62e073',
  'tags': ['nucleus', 'RNA']},
 {'_id': '6501d9b127b4271da176f2f2',
  'access': {'groups': [],
   'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
  'childId': '64ff2f7de7761ae25b62e0a6',
  'datasetId': '64ff0d8ce7761ae25b62da75',
  'parentId': '64ff2f64e7761ae25b62e073',
  'tags': ['nucleus', 'RNA']},
 {'_id': '6501d9b127b4271da176f2f3',
  'access': {'groups': [],
   'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
  'childId': '64ff2f7ee7761ae25b62e0a9',
  'datasetId': '64ff0d8ce7761ae25b62da75',
  'parentId': '64ff2f64e7761ae25b62e073',
  'tags': ['nucleus', 'RNA']},
 {'_id': '6501d9b127b4271da176f2f4',
  'access': {'groups': [],
   'users': [{'flags': [], 'id': '62891fc0206

In [48]:
# OLD
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial import cKDTree
import numpy as np

def extract_spatial_annotation_data(obj_list):
    data = []
    for obj in obj_list:
        x, y = None, None
        shape = obj['shape']
        coords = obj['coordinates']
        
        if shape == 'point':
            x, y = coords[0]['x'], coords[0]['y']
        elif shape == 'polygon':
            polygon = Polygon([(pt['x'], pt['y']) for pt in coords])
            centroid = polygon.centroid
            x, y = centroid.x, centroid.y
        
        data.append({
            '_id': obj['_id'],
            'x': x,
            'y': y,
            'Time': obj['location']['Time'],
            'XY': obj['location']['XY'],
            'Z': obj['location']['Z']
        })
    return data

# Extract data and create dataframes
child_data = extract_spatial_annotation_data(childList)
parent_data = extract_spatial_annotation_data(parentList)

child_df = pd.DataFrame(child_data)
parent_df = pd.DataFrame(parent_data)

gdf_child = gpd.GeoDataFrame(child_df, geometry=gpd.points_from_xy(child_df.x, child_df.y))
gdf_parent = gpd.GeoDataFrame(parent_df, geometry=gpd.points_from_xy(parent_df.x, parent_df.y))

# Create cKDTree object to represent the nucleus data points
tree = cKDTree(np.array(list(zip(gdf_parent.geometry.x, gdf_parent.geometry.y))))

# Compute distance and index for the nearest nucleus for each spot
distances, indices = tree.query(np.array(list(zip(gdf_child.geometry.x, gdf_child.geometry.y))))

# Map spot IDs to nearest nucleus IDs
child_to_parent = pd.DataFrame({
    'spot_id': child_df['_id'],
    'nearest_nucleus_id': parent_df.iloc[indices]['_id'].values
})

print(len(child_to_parent))


36


In [14]:
# OLD
myNewConnections = []
combined_tags = list(set([parentTag, childTag]))

for index, row in point_to_polygon.iterrows():
    point_id = row['point_id']
    polygon_id = row['nearest_polygon_id']
    
    myNewConnections.append({
        'datasetId': datasetId,  # assuming you've already set this variable elsewhere in your code
        'parentId': polygon_id,
        'childId': point_id,
        'tags': combined_tags
    })

pprint.pprint(myNewConnections,indent=2)
# for connection in myNewConnections:
#     annotationClient.createConnection(connection)

annotationClient.createMultipleConnections(myNewConnections)


[ { 'childId': '64fa63e446017e1063ec2969',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63d946017e1063ec2957',
    'tags': ['myPoint', 'myBlob']},
  { 'childId': '64fa63e546017e1063ec296c',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63da46017e1063ec295a',
    'tags': ['myPoint', 'myBlob']},
  { 'childId': '64fa63e646017e1063ec296f',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63dc46017e1063ec295d',
    'tags': ['myPoint', 'myBlob']},
  { 'childId': '64fa63e746017e1063ec2972',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63e146017e1063ec2966',
    'tags': ['myPoint', 'myBlob']},
  { 'childId': '64fa63e846017e1063ec2975',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63e046017e1063ec2963',
    'tags': ['myPoint', 'myBlob']},
  { 'childId': '64fa63e946017e1063ec2978',
    'datasetId': '64fa638546017e1063ec2941',
    'parentId': '64fa63de46017e1063ec2960',
    'tags': ['myPoint', 'myB

[{'_id': '650083e027b4271da176f1a6',
  'access': {'groups': [],
   'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
  'childId': '64fa63e446017e1063ec2969',
  'datasetId': '64fa638546017e1063ec2941',
  'parentId': '64fa63d946017e1063ec2957',
  'tags': ['myPoint', 'myBlob']},
 {'_id': '650083e027b4271da176f1a7',
  'access': {'groups': [],
   'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
  'childId': '64fa63e546017e1063ec296c',
  'datasetId': '64fa638546017e1063ec2941',
  'parentId': '64fa63da46017e1063ec295a',
  'tags': ['myPoint', 'myBlob']},
 {'_id': '650083e027b4271da176f1a8',
  'access': {'groups': [],
   'users': [{'flags': [], 'id': '62891fc02060d362984bc104', 'level': 2}]},
  'childId': '64fa63e646017e1063ec296f',
  'datasetId': '64fa638546017e1063ec2941',
  'parentId': '64fa63dc46017e1063ec295d',
  'tags': ['myPoint', 'myBlob']},
 {'_id': '650083e027b4271da176f1a9',
  'access': {'groups': [],
   'users': [{'flags': [], 'id': '62